#### Before we get the data and start exploring it, let's download all the dependencies that we will need.

In [2]:
import pandas as pd
import numpy as np
import requests
import csv

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# import k-means from clustering stage
from sklearn.cluster import KMeans

!conda install -c conda-forge folium=0.5.0 --yes 
import folium # map rendering library

Fetching package metadata .............
Solving package specifications: .

Package plan for installation in environment /opt/conda/envs/DSX-Python35:

The following NEW packages will be INSTALLED:

    altair:  2.2.2-py35_1 conda-forge
    branca:  0.3.1-py_0   conda-forge
    folium:  0.5.0-py_0   conda-forge
    vincent: 0.4.4-py_1   conda-forge

altair-2.2.2-p 100% |################################| Time: 0:00:00  51.46 MB/s
branca-0.3.1-p 100% |################################| Time: 0:00:00  34.95 MB/s
vincent-0.4.4- 100% |################################| Time: 0:00:00  39.67 MB/s
folium-0.5.0-p 100% |################################| Time: 0:00:00  46.93 MB/s


#### Tranform the data into a *pandas* dataframe

In [3]:
# The code was removed by Watson Studio for sharing.

,Area,Latitude,Longitude
0,Klisa,45.302926,19.823393
1,Slana Bara,45.288153,19.828563
2,Vidovdansko Naselje,45.280669,19.832978
3,Salajka,45.270604,19.836438
4,Podbara,45.262004,19.850332


#### Define Foursquare Credentials and Version

In [4]:
CLIENT_ID = 'R4HPVE2EPLS5EJD03UM5IPWWNMETZWJUEBZCGHZYEZGLBVJN'
CLIENT_SECRET = '0M1EO4BLBQRCDWR2FOX3OH21FJKIF33G34U3DEWJB4LJIA3Q'
VERSION = '20180605' # Foursquare API version


print('My credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

My credentails:
CLIENT_ID: R4HPVE2EPLS5EJD03UM5IPWWNMETZWJUEBZCGHZYEZGLBVJN
CLIENT_SECRET:0M1EO4BLBQRCDWR2FOX3OH21FJKIF33G34U3DEWJB4LJIA3Q


#### Let's create a function to repeat the same process to all the neighborhoods in Novi Sad

In [5]:
def getNearbyVenues(names, latitudes, longitudes, radius=1000):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Area', 
                  'Area Latitude', 
                  'Area Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

#### Now write the code to run the above function on each neighborhood and create a new dataframe called *df_venues*.

In [6]:
df_venues = getNearbyVenues(names=df_areas['Area'],
                                   latitudes=df_areas['Latitude'],
                                   longitudes=df_areas['Longitude']
                                  )
df_venues.head()

Klisa
Slana Bara
Vidovdansko Naselje
Salajka
Podbara
Stari Grad
Rotkvarija
Banatic
Grbavica
Adamovicevo Naselje
Sajmiste
Detelinara
Avijaticarsko Naselje
Bistrica
Telep
Adice
Liman 1
Liman 2
Liman 3
Liman 4
Jugovicevo
Sajlovo


,Area,Area Latitude,Area Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Klisa,45.302926,19.823393,Sole Mio ****,45.310429,19.824550,Hotel
1,Klisa,45.302926,19.823393,Okretnica Jedinice,45.302245,19.823976,Bus Station
2,Klisa,45.302926,19.823393,Roda Megamarket,45.305172,19.833761,Supermarket
3,Klisa,45.302926,19.823393,Kaćka petlja,45.306639,19.826250,Road
4,Klisa,45.302926,19.823393,Milka Z,45.301300,19.829960,Hotel


In [7]:
df_area_venue_category =  df_venues.drop(columns=['Area Latitude', 'Area Longitude', 'Venue', 'Venue Latitude', 'Venue Longitude'])
df_area_venue_category

,Area,Venue Category
0,Klisa,Hotel
1,Klisa,Bus Station
2,Klisa,Supermarket
3,Klisa,Road
4,Klisa,Hotel
5,Klisa,Fast Food Restaurant
6,Klisa,Brewery
7,Klisa,Department Store
8,Slana Bara,Indonesian Restaurant
9,Slana Bara,Pool


In [8]:
df_area_venue_category_count = df_area_venue_category.groupby(['Area', 'Venue Category']).size().reset_index(name='Counts')
df_area_venue_category_count

,Area,Venue Category,Counts
0,Adamovicevo Naselje,BBQ Joint,1
1,Adamovicevo Naselje,Bakery,2
2,Adamovicevo Naselje,Bar,1
3,Adamovicevo Naselje,Bistro,1
4,Adamovicevo Naselje,Café,5
5,Adamovicevo Naselje,Coffee Shop,1
6,Adamovicevo Naselje,Cosmetics Shop,2
7,Adamovicevo Naselje,Creperie,1
8,Adamovicevo Naselje,Gym,1
9,Adamovicevo Naselje,Hotel,4


In [9]:
df_area_venue_category_count.loc[df_area_venue_category_count['Venue Category'] == 'Gym']

,Area,Venue Category,Counts
8,Adamovicevo Naselje,Gym,1
27,Avijaticarsko Naselje,Gym,1
45,Banatic,Gym,1
80,Detelinara,Gym,1
99,Grbavica,Gym,1
238,Rotkvarija,Gym,2
261,Sajmiste,Gym,2
307,Stari Grad,Gym,1
324,Telep,Gym,1


#### Analyze Each Neighborhood

In [13]:
# one hot encoding
novisad_onehot = pd.get_dummies(df_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
novisad_onehot['Neighborhood'] = df_venues['Area'] 

# move neighborhood column to the first column
fixed_columns = [novisad_onehot.columns[-1]] + list(novisad_onehot.columns[:-1])
novisad_onehot = novisad_onehot[fixed_columns]

novisad_onehot.head()

,Neighborhood,Arcade,Art Gallery,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Beach Bar,...,Sporting Goods Shop,Sports Bar,Supermarket,Swiss Restaurant,Tennis Court,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court
0,Klisa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,Klisa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,Klisa,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
3,Klisa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,Klisa,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


#### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [14]:
novisad_grouped = novisad_onehot.groupby('Neighborhood').mean().reset_index()
novisad_grouped

,Neighborhood,Arcade,Art Gallery,Art Museum,Athletics & Sports,BBQ Joint,Bakery,Bar,Beach,Beach Bar,...,Sporting Goods Shop,Sports Bar,Supermarket,Swiss Restaurant,Tennis Court,Theater,Toy / Game Store,Train Station,Vegetarian / Vegan Restaurant,Volleyball Court
0,Adamovicevo Naselje,0.000000,0.000000,0.000000,0.000000,0.033333,0.066667,0.033333,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,Adice,0.000000,0.000000,0.000000,0.200000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
2,Avijaticarsko Naselje,0.000000,0.000000,0.000000,0.000000,0.000000,0.166667,0.000000,0.083333,0.000000,...,0.000000,0.000000,0.083333,0.083333,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
3,Banatic,0.000000,0.000000,0.000000,0.000000,0.000000,0.100000,0.066667,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
4,Bistrica,0.000000,0.000000,0.000000,0.000000,0.033333,0.033333,0.066667,0.000000,0.000000,...,0.000000,0.000000,0.100000,0.033333,0.033333,0.000000,0.033333,0.000000,0.000000,0.000000
5,Detelinara,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.066667,0.000000,0.000000,...,0.000000,0.000000,0.066667,0.033333,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000
6,Grbavica,0.000000,0.000000,0.000000,0.000000,0.066667,0.100000,0.033333,0.000000,0.000000,...,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.000000,0.000000,0.066667,0.000000
7,Jugovicevo,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
8,Klisa,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.125000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,Liman 1,0.000000,0.000000,0.000000,0.033333,0.000000,0.000000,0.000000,0.066667,0.033333,...,0.000000,0.033333,0.066667,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000


In [17]:
num_top_venues = 5

for hood in novisad_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = novisad_grouped[novisad_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adamovicevo Naselje----
                venue  freq
0                Café  0.17
1               Hotel  0.13
2  Italian Restaurant  0.10
3              Bakery  0.07
4      Cosmetics Shop  0.07


----Adice----
                venue  freq
0              Resort   0.2
1  Athletics & Sports   0.2
2               River   0.2
3  Seafood Restaurant   0.2
4         Music Venue   0.2


----Avijaticarsko Naselje----
                  venue  freq
0                Bakery  0.17
1  Gym / Fitness Center  0.08
2      Swiss Restaurant  0.08
3           Pizza Place  0.08
4     Food & Drink Shop  0.08


----Banatic----
         venue  freq
0  Pizza Place  0.10
1         Café  0.10
2       Bakery  0.10
3       Lounge  0.07
4   Restaurant  0.07


----Bistrica----
                    venue  freq
0              Restaurant  0.17
1             Supermarket  0.10
2  Furniture / Home Store  0.10
3                    Café  0.10
4                     Bar  0.07


----Detelinara----
            venue  freq
0      R

In [18]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

#### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [21]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = novisad_grouped['Neighborhood']

for ind in np.arange(novisad_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(novisad_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Adamovicevo Naselje,Café,Hotel,Italian Restaurant,Cosmetics Shop,Bakery,Spa,Park,Creperie,Pizza Place,Coffee Shop
1,Adice,Resort,Athletics & Sports,River,Music Venue,Seafood Restaurant,Volleyball Court,Event Space,Creperie,Cultural Center,Deli / Bodega
2,Avijaticarsko Naselje,Bakery,Gym / Fitness Center,Supermarket,Pizza Place,Soccer Field,Food & Drink Shop,Beach,Market,Swiss Restaurant,Gas Station
3,Banatic,Café,Pizza Place,Bakery,Lounge,Sandwich Place,Restaurant,Fast Food Restaurant,Burger Joint,Bar,Italian Restaurant
4,Bistrica,Restaurant,Furniture / Home Store,Supermarket,Café,Grocery Store,Bar,Gym / Fitness Center,Salad Place,Hotel,Fast Food Restaurant


#### Cluster Neighborhoods

In [23]:
# set number of clusters
kclusters = 5

novisad_grouped_clustering = novisad_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(novisad_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 4, 0, 0, 0, 0, 0, 1, 3, 0], dtype=int32)

In [ ]:
# add clustering labels
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

In [31]:
novisad_merged = df_areas

novisad_merged = novisad_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Area')

novisad_merged.head() # check the last columns!

,Area,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Klisa,45.302926,19.823393,3,Hotel,Department Store,Supermarket,Road,Bus Station,Brewery,Fast Food Restaurant,Volleyball Court,Farmers Market,Cultural Center
1,Slana Bara,45.288153,19.828563,0,Volleyball Court,Indonesian Restaurant,Pool,Soccer Field,Flea Market,Comfort Food Restaurant,Event Space,Cosmetics Shop,Creperie,Cultural Center
2,Vidovdansko Naselje,45.280669,19.832978,0,Clothing Store,Furniture / Home Store,Café,Electronics Store,Boutique,Fast Food Restaurant,Bookstore,Shopping Mall,Multiplex,Pet Store
3,Salajka,45.270604,19.836438,0,Clothing Store,Bakery,Café,Furniture / Home Store,Hotel,Supermarket,Restaurant,Boutique,Gas Station,Fast Food Restaurant
4,Podbara,45.262004,19.850332,0,Bar,Restaurant,Eastern European Restaurant,Italian Restaurant,Pub,Ice Cream Shop,Lounge,Library,Café,Coffee Shop


In [33]:
latitude = 45.25167
longitude = 19.83694

#### Finally, let's visualize the resulting clusters

In [35]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(novisad_merged['Latitude'], novisad_merged['Longitude'], novisad_merged['Area'], novisad_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

#### Examine Clusters

In [38]:
novisad_merged.loc[novisad_merged['Cluster Labels'] == 0, novisad_merged.columns[[0] + list(range(4, novisad_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Slana Bara,Volleyball Court,Indonesian Restaurant,Pool,Soccer Field,Flea Market,Comfort Food Restaurant,Event Space,Cosmetics Shop,Creperie,Cultural Center
2,Vidovdansko Naselje,Clothing Store,Furniture / Home Store,Café,Electronics Store,Boutique,Fast Food Restaurant,Bookstore,Shopping Mall,Multiplex,Pet Store
3,Salajka,Clothing Store,Bakery,Café,Furniture / Home Store,Hotel,Supermarket,Restaurant,Boutique,Gas Station,Fast Food Restaurant
4,Podbara,Bar,Restaurant,Eastern European Restaurant,Italian Restaurant,Pub,Ice Cream Shop,Lounge,Library,Café,Coffee Shop
5,Stari Grad,Restaurant,Plaza,Lounge,Bar,Coffee Shop,Sandwich Place,Brewery,Gym,Hotel,Italian Restaurant
6,Rotkvarija,Bar,Restaurant,Café,Lounge,Gym,Coffee Shop,Plaza,Ice Cream Shop,Library,Italian Restaurant
7,Banatic,Café,Pizza Place,Bakery,Lounge,Sandwich Place,Restaurant,Fast Food Restaurant,Burger Joint,Bar,Italian Restaurant
8,Grbavica,Café,Bakery,BBQ Joint,Pizza Place,Park,Gym / Fitness Center,Vegetarian / Vegan Restaurant,Farmers Market,Sandwich Place,Dessert Shop
9,Adamovicevo Naselje,Café,Hotel,Italian Restaurant,Cosmetics Shop,Bakery,Spa,Park,Creperie,Pizza Place,Coffee Shop
10,Sajmiste,Bar,Café,Restaurant,Italian Restaurant,Gym,Sandwich Place,Hotel,Snack Place,Lounge,Coffee Shop


In [39]:
novisad_merged.loc[novisad_merged['Cluster Labels'] == 1, novisad_merged.columns[[0] + list(range(4, novisad_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
20,Jugovicevo,Restaurant,Volleyball Court,Fast Food Restaurant,Creperie,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Eastern European Restaurant,Electronics Store


In [40]:
novisad_merged.loc[novisad_merged['Cluster Labels'] == 2, novisad_merged.columns[[0] + list(range(4, novisad_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
21,Sajlovo,Gym / Fitness Center,Restaurant,Athletics & Sports,Fast Food Restaurant,Creperie,Cultural Center,Deli / Bodega,Department Store,Dessert Shop,Eastern European Restaurant


In [41]:
novisad_merged.loc[novisad_merged['Cluster Labels'] == 3, novisad_merged.columns[[0] + list(range(4, novisad_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Klisa,Hotel,Department Store,Supermarket,Road,Bus Station,Brewery,Fast Food Restaurant,Volleyball Court,Farmers Market,Cultural Center


In [42]:
novisad_merged.loc[novisad_merged['Cluster Labels'] == 4, novisad_merged.columns[[0] + list(range(4, novisad_merged.shape[1]))]]

,Area,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
15,Adice,Resort,Athletics & Sports,River,Music Venue,Seafood Restaurant,Volleyball Court,Event Space,Creperie,Cultural Center,Deli / Bodega


## **The Battle of Neighborhoods in Novi Sad**

Business Problem: recommendation of the best places in the city Novi Sad to open a business.

Data: We will use a csv file with list of all areas in Novi Sad and their coordinates which help us to use 
Foursquare. We generated this csv file with data from Wikipedia.

Explanation of work flow:

- Download all the dependencies that we will need.
- Tranform the data into a pandas dataframe
- Define Foursquare Credentials and Version
- Create a function to repeat the same process to all the neighborhoods in Novi Sad
- Create a new dataframe called df_venues filled using the above function on each neighborhood
- Analyze Each Neighborhood
- Group rows by neighborhood and by taking the mean of the frequency of occurrence of each category
- Create the new dataframe and display the top 10 venues for each neighborhood
- Cluster Neighborhoods using 5 as number of clusters and methodology K-means
- Visualize the resulting clusters

Recommendation: 

After coding to get the best options, we got the following conclusions:

- If someone is looking to open a restaurant the best areas are: Salajka and Grbavica
- If someone is looking to open a spa the best areas are all except Adamovicevo Naselje
- If someone is looking to open a gym the best areas are all except Stari Grad